Kernel PCA is also capable of denoising data.

In [1]:
from src.kernel_pca.kpca import KernelPCA
from src.kernel_pca.kernels import RBFKernel
from src.pca import PCA

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

import numpy as np

import matplotlib.pyplot as plt

ImportError: attempted relative import with no known parent package

In [ ]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)